In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
def print_partitions(ddf):
    for i in range(ddf.npartitions):
        print(ddf.partitions[i].compute())

In [3]:
df = pd.DataFrame(
    {"col1": ["aa", "dd", "cc", "bb", "aa"], "col2": ["a1", "d", "c", "b", "a2"]}
)
ddf = dd.from_pandas(df, npartitions=2)

In [4]:
print(ddf.compute())

  col1 col2
0   aa   a1
1   dd    d
2   cc    c
3   bb    b
4   aa   a2


In [5]:
ddf.dtypes

col1    object
col2    object
dtype: object

In [6]:
ddf.divisions

(0, 3, 4)

In [7]:
ddf.known_divisions

True

In [8]:
print_partitions(ddf)

  col1 col2
0   aa   a1
1   dd    d
2   cc    c
  col1 col2
3   bb    b
4   aa   a2


In [9]:
ddf2 = ddf.set_index("col1", divisions=["aa", "cc", "dd"])

In [10]:
print(ddf2.compute())

     col2
col1     
aa     a1
aa     a2
bb      b
cc      c
dd      d


In [11]:
ddf2.divisions

('aa', 'cc', 'dd')

In [12]:
print_partitions(ddf2)

     col2
col1     
aa     a1
aa     a2
bb      b
     col2
col1     
cc      c
dd      d


## Cluster benchmarking

In [2]:
import coiled
import dask

In [3]:
cluster = coiled.Cluster(name="powers-cluster2", n_workers=5)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/rich/live.py:226: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


Found software environment build
Created fw rule: inbound [22-22] [0.0.0.0/0] []
Created fw rule: inbound [8786-8786] [0.0.0.0/0] []
Created fw rule: inbound [8787-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-matthew24-78862-firewall
Created fw rule: inbound [0-65535] [None] [coiled-dask-matthew24-78862-firewall]
Created scheduler VM: coiled-dask-matthew24-78862-scheduler (type: t3.medium, ip: ['44.193.29.105'])


In [4]:
client = dask.distributed.Client(cluster)

In [5]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [13]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930


In [6]:
ddf.known_divisions

True

In [7]:
ddf.divisions

(Timestamp('2000-01-01 00:00:00'),
 Timestamp('2000-01-08 00:00:00'),
 Timestamp('2000-01-15 00:00:00'),
 Timestamp('2000-01-22 00:00:00'),
 Timestamp('2000-01-29 00:00:00'),
 Timestamp('2000-02-05 00:00:00'),
 Timestamp('2000-02-12 00:00:00'),
 Timestamp('2000-02-19 00:00:00'),
 Timestamp('2000-02-26 00:00:00'),
 Timestamp('2000-03-04 00:00:00'),
 Timestamp('2000-03-11 00:00:00'),
 Timestamp('2000-03-18 00:00:00'),
 Timestamp('2000-03-25 00:00:00'),
 Timestamp('2000-04-01 00:00:00'),
 Timestamp('2000-04-08 00:00:00'),
 Timestamp('2000-04-15 00:00:00'),
 Timestamp('2000-04-22 00:00:00'),
 Timestamp('2000-04-29 00:00:00'),
 Timestamp('2000-05-06 00:00:00'),
 Timestamp('2000-05-13 00:00:00'),
 Timestamp('2000-05-20 00:00:00'),
 Timestamp('2000-05-27 00:00:00'),
 Timestamp('2000-06-03 00:00:00'),
 Timestamp('2000-06-10 00:00:00'),
 Timestamp('2000-06-17 00:00:00'),
 Timestamp('2000-06-24 00:00:00'),
 Timestamp('2000-07-01 00:00:00'),
 Timestamp('2000-07-08 00:00:00'),
 Timestamp('2000-07-

In [12]:
len(ddf.divisions)

1096

In [19]:
%%time

len(ddf.loc["2000-01-01 00:00:02":"2000-01-01 00:00:04"])

CPU times: user 76.4 ms, sys: 8.8 ms, total: 85.2 ms
Wall time: 2.54 s


3

In [30]:
ddf2 = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
    index=False,
)

In [31]:
ddf2.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
1,2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2,2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
3,2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
4,2000-01-01 00:00:04,967,Michael,-0.251460,0.810930


In [33]:
%%time

len(
    ddf2.loc[
        (ddf2["timestamp"] >= "2000-01-01 00:00:02")
        & (ddf2["timestamp"] <= "2000-01-01 00:00:04")
    ]
)

CPU times: user 380 ms, sys: 46.1 ms, total: 426 ms
Wall time: 1min 55s


3

In [19]:
ddf3 = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
    index=False,
)

In [20]:
ddf3.npartitions

1095

In [16]:
%%time

ddf3.loc[ddf3["id"] >= 1001].name.nunique().compute()

KeyboardInterrupt: 

In [45]:
%%time

ddf3.set_index("timestamp", sorted=True)

CPU times: user 1.83 s, sys: 138 ms, total: 1.97 s
Wall time: 1min 46s


,id,name,x,y
npartitions=1095,,,,
2000-01-01 00:00:00,int64,object,float64,float64
2000-01-08 00:00:00,...,...,...,...
...,...,...,...,...
2020-12-19 00:00:00,...,...,...,...
2020-12-25 23:59:59,...,...,...,...


In [9]:
id_min, id_max = dask.compute(ddf.id.min(), ddf.id.max())

In [10]:
id_min

815

In [11]:
id_max

1193

## Figuring out our own divisions

In [21]:
dask_computed_divisions = ddf3.set_index("id").divisions

In [22]:
unique = list(dict.fromkeys(list(dask_computed_divisions)))

In [25]:
len(unique)

149

In [26]:
%%time

ddf3u = ddf3.set_index("id", divisions=unique)

CPU times: user 33 ms, sys: 6.2 ms, total: 39.2 ms
Wall time: 37.5 ms


In [29]:
ddf3u.npartitions

148

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLEOFError: EOF occurred in violation of protocol (_ssl.c:1129)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/

In [27]:
%%time

ddf3u.loc[[1001]].name.nunique().compute()

CPU times: user 952 ms, sys: 92.4 ms, total: 1.04 s
Wall time: 2min 10s


26

## Use Dask computed divisions

In [10]:
%%time

ddf3a = ddf3.set_index("id")

CPU times: user 3.01 s, sys: 344 ms, total: 3.36 s
Wall time: 2min 14s


In [11]:
%%time

ddf3a.loc[[1001]].name.nunique().compute()

CPU times: user 635 ms, sys: 49.9 ms, total: 685 ms
Wall time: 1min 58s


26

## Warning: the next computation will cause your cluster to blow up (it'll run out of memory)

In [25]:
%%time

ddf3b = ddf3.set_index("id", divisions=[800, 900, 1000, 1100, 1200])

CPU times: user 25.5 ms, sys: 4.09 ms, total: 29.6 ms
Wall time: 27.7 ms


In [26]:
%%time

ddf3b.loc[[1001]].name.nunique().compute()

KeyboardInterrupt: 

In [47]:
%%time

ddf3.set_index("id", sorted=True)

ValueError: ('Partitions must be sorted ascending with the index', [858, 862, 863, 843, 850, 855, 852, 852, 846, 846, 860, 854, 847, 847, 864, 850, 860, 848, 859, 863, 843, 856, 857, 850, 859, 861, 853, 856, 853, 856, 851, 850, 864, 842, 862, 856, 857, 854, 852, 847, 844, 849, 832, 852, 855, 863, 850, 857, 844, 857, 854, 861, 853, 855, 837, 840, 839, 865, 860, 841, 836, 859, 849, 855, 861, 853, 863, 854, 845, 849, 853, 858, 850, 851, 833, 843, 848, 854, 839, 836, 862, 858, 857, 858, 845, 855, 846, 844, 850, 856, 861, 861, 866, 849, 857, 856, 863, 848, 858, 860, 849, 854, 867, 859, 836, 855, 843, 867, 866, 867, 858, 855, 853, 856, 856, 853, 863, 856, 856, 841, 860, 852, 851, 855, 868, 850, 852, 819, 859, 859, 854, 841, 858, 855, 829, 859, 853, 856, 854, 859, 850, 853, 855, 852, 839, 864, 842, 863, 853, 844, 863, 852, 858, 864, 843, 851, 861, 848, 846, 857, 857, 845, 861, 850, 854, 845, 853, 856, 850, 849, 863, 850, 860, 853, 857, 848, 859, 858, 853, 858, 845, 860, 853, 861, 848, 862, 85

In [33]:
2 + 2

4